In [1]:
import nidaqmx
from time import ctime, time
import numpy as np
import csv
import nidaqmx.system
from nidaqmx.constants import *
from rich import print

In [2]:
system = nidaqmx.system.System.local()
devices = system.devices

In [3]:
print("장치를 선택해주세요")
for idx, device in enumerate(devices):
    print(idx, ']', device.name)

selected = int(input())
device = devices[selected]

장치를 선택해주세요

0 ] cDAQ1

1 ] cDAQ1Mod1

1


In [ ]:
task = nidaqmx.Task()
print('add input channels')

channels = device.ai_physical_chans
for idx, channel in enumerate(channels):
    print("channel ",idx , ": ", channel.name)
    task.ai_channels.add_ai_voltage_chan(channel.name)
    
print()

In [19]:
task = nidaqmx.Task()
channels = input("채널 이름을 입력해 주세요 ex) ai0:3 : ")
task.ai_channels.add_ai_voltage_chan(device.name + "/" + channels)

채널 이름을 입력해 주세요 ex) ai0:3 : ai0:2


AIChannel(name=cDAQ1Mod1/ai0:2)

In [20]:
task.timing.cfg_samp_clk_timing(rate=51200,
                                active_edge=nidaqmx.constants.Edge.RISING,
                                sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
                                samps_per_chan=51200)
print('task ready')

task ready

In [22]:
for t in device.ai_physical_chans:
    print(t)

PhysicalChannel(name=cDAQ1Mod1/ai0)

PhysicalChannel(name=cDAQ1Mod1/ai1)

PhysicalChannel(name=cDAQ1Mod1/ai2)

PhysicalChannel(name=cDAQ1Mod1/ai3)

In [23]:
filePath = input("저장할 파일명을 입력해주세요")
    
writerList = []
fileList = []
for channel in task.channels:
    name = filePath +"_"+ channel.name.replace("/", "_") + ".csv"
    print('name = ', name)
    fileList.append(open(name, 'a', newline='\n'))
    writerList.append(csv.writer(fileList[-1]))

저장할 파일명을 입력해주세요data/test222


name =  data/test222_cDAQ1Mod1_ai0.csv

name =  data/test222_cDAQ1Mod1_ai1.csv

name =  data/test222_cDAQ1Mod1_ai2.csv

In [21]:
while True:
    datas = task.read(number_of_samples_per_channel=-1)
    for idx, data in enumerate(datas):
        print(len(data))
        for raw in data:
            writerList[idx].writerow([ctime(time()), raw])

51200

51200

51200

51200

51200

...

51200

51200

51200

51200

KeyboardInterrupt: 

In [22]:
for f in fileList:
    f.close()

In [8]:
task.stop()

C:\develop\anaconda\lib\site-packages\nidaqmx\errors.py:206: DaqWarning: 
Warning 200010 occurred.

Finite acquisition or generation has been stopped before the requested number of samples were acquired or generated.
  warnings.warn(DaqWarning(


In [24]:
task.ai_channels.add_ai_rtd_chan("cDAQ2Mod3/ai0", min_val=0.0, max_val=100.0, rtd_type=RTDType.PT_3750, resistance_config=ResistanceConfiguration.THREE_WIRE, current_excit_source=.ExcitationSource.INTERNAL, current_excit_val=0.00100)

DaqError: Device identifier is invalid.
Device Specified: cDAQ2Mod3
Suggested Device(s): cDAQ1, cDAQ1Mod1

Task Name: _unnamedTask<1>

Status Code: -200220

In [24]:
import nidaqmx
from time import ctime, time
import numpy as np
import csv
import nidaqmx.system
from nidaqmx.constants import *

system = nidaqmx.system.System.local()
devices = system.devices

print("장치를 선택해주세요")
for idx, device in enumerate(devices):
    print(idx, ']', device.name)

selected = int(input())
device = devices[selected]

task = nidaqmx.Task()
print('add input channels')

channels = device.ai_physical_chans
for channel in channels:
    print("channel : ", channel.name)

channels = input("채널 이름을 입력해 주세요 ex) ai0:3 : ")
task.ai_channels.add_ai_voltage_chan(device.name + "/" + channels)

task.timing.cfg_samp_clk_timing(rate=51200,
                                active_edge=nidaqmx.constants.Edge.RISING,
                                sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
                                samps_per_chan=51200)
print('task ready')

filePath = input("저장할 파일명을 입력해주세요")
    
writerList = []
fileList = []
for channel in task.channels:
    name = filePath +"_"+ channel.name.replace("/", "_") + ".csv"
    print('name = ', name)
    fileList.append(open(name, 'a', newline='\n'))
    writerList.append(csv.writer(fileList[-1]))
    
while True:
    datas = task.read(number_of_samples_per_channel=-1)
    for idx, data in enumerate(datas):
        print(len(data))
        for raw in data:
            writerList[idx].writerow([ctime(time()), raw])

장치를 선택해주세요

0 ] cDAQ1

1 ] cDAQ1Mod1

1


add input channels

...

51200

51200

51200

51200

KeyboardInterrupt: 